# Introduksjon

## Forfatter(e)
1. Magne Furuholmen Myhren

## Bestilling

Epost fra Terje Landsem, K 425 Seksjon for energi, miljø- og transport 

```
Hei igjen,

Skulle hatt noe data på årsverk for helsetjenester, (årsverk for nace: 86.101, 86.102 og 86.906) til statistikken over farlig avfall vi jobber med. Det er tilsvarende tall for som for 2020, for 2021-årgangen vi behøver for farlig avfall. Helst så fort som mulig 😊

Mvh Terje Landsem
```


# Setup

## Velger årgang

In [ ]:
aargang = 2021

# Næringskoder som skal med
naring = ["86.101", "86.102", "86.906"]

# Kolonner fra sysskostra
cols = ["VIRK_ID_SSB", "VIRK_NAVN", "FRTK_ID_SSB", "FRTK_NAVN", "VIRK_NACE1_SN07", "ARB_AARSVERK", "PERS_LANGEFRA_ANDEL", "VIRK_HO_MERKE"]

## Importerer pakker og setter innstillinger

In [ ]:
import numpy as np
import pandas as pd

# Fjerner begrensning på antall rader og kolonner som vises av gangen
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

# Unngå standardform i output
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Laster inn personell rådata for personelldata (sysskostra)

In [ ]:
sti = f"/ssb/stamme01/helsep/wk24/g{str(aargang)}_sysskostra.parquet"
df = pd.read_parquet(sti)

# Virksomheter markert med SPES(helse)

In [ ]:
df1 = df[cols].copy()

# Formel for å regne ut årsverk. (Trekker fra langtidsfravær)
df1['ARSVERK_EKSKL'] = df1['ARB_AARSVERK']-(df1['ARB_AARSVERK']*df1['PERS_LANGEFRA_ANDEL'])

In [ ]:
df1 = df1.loc[df1['VIRK_NACE1_SN07'].isin(naring)]
df1 = df1.loc[df1['VIRK_HO_MERKE'].isin(["SPES"])]

In [ ]:
df1 = df1.drop(columns=['ARB_AARSVERK','PERS_LANGEFRA_ANDEL'])

virk_i_delreg = df1.groupby(['VIRK_ID_SSB','VIRK_NAVN','FRTK_ID_SSB','FRTK_NAVN','VIRK_NACE1_SN07']).sum()
virk_i_delreg = virk_i_delreg.reset_index()

virk_i_delreg = virk_i_delreg.rename(columns = {'ARSVERK_EKSKL': 'SUM_ARSVERK_EKSKL'})
virk_i_delreg['SUM_ARSVERK_EKSKL'] = virk_i_delreg['SUM_ARSVERK_EKSKL'].astype(str).apply(lambda x: x.replace(".", ","))

In [ ]:
virk_i_delreg.head(5)

In [ ]:
virk_i_delreg.shape

# Virksomheter ikke markert med SPES(helse)

In [ ]:
df1 = df[cols].copy()

# Formel for å regne ut årsverk. (Trekker fra langtidsfravær)
df1['ARSVERK_EKSKL'] = df1['ARB_AARSVERK']-(df1['ARB_AARSVERK']*df1['PERS_LANGEFRA_ANDEL'])

In [ ]:
df1 = df1.loc[df1['VIRK_NACE1_SN07'].isin(naring)]
df1 = df1.loc[~df1['VIRK_HO_MERKE'].isin(["SPES"])]

In [ ]:
df1 = df1.drop(columns=['ARB_AARSVERK','PERS_LANGEFRA_ANDEL'])

virk_ikke_i_delreg = df1.groupby(['VIRK_ID_SSB','VIRK_NAVN','FRTK_ID_SSB','FRTK_NAVN','VIRK_NACE1_SN07']).sum()
virk_ikke_i_delreg = virk_ikke_i_delreg.reset_index()

virk_ikke_i_delreg = virk_ikke_i_delreg.rename(columns = {'ARSVERK_EKSKL': 'SUM_ARSVERK_EKSKL'})
virk_ikke_i_delreg['SUM_ARSVERK_EKSKL'] = virk_ikke_i_delreg['SUM_ARSVERK_EKSKL'].astype(str).apply(lambda x: x.replace(".", ","))

In [ ]:
virk_ikke_i_delreg.head(5)

In [ ]:
virk_ikke_i_delreg.shape

# Eksportering til .csv-filer og lagring
Filene lagres i samme mappe som dette .ipynb-dokumentet

In [ ]:
def lagre_fil(skj):
    dato_idag = pd.Timestamp("today").strftime("%d%m%y")
    filnavn = "aarsverk_etter_nace_" + skj + "_" + str(aargang) + "_" + dato_idag + ".csv"
    eval(skj).to_csv(filnavn, sep=";", encoding='latin1', index=False)
    print(filnavn, " lagret")

In [ ]:
lagre_fil("virk_i_delreg")
lagre_fil("virk_ikke_i_delreg")